# 🎓 Conference Paper Assignment with AI

## The Challenge

You're organizing an AI conference with **15 accepted papers** and **6 sessions** (oral and poster).

**The complexity:**
- 📊 Oral sessions require scores ≥ 4.0
- 🏷️ Papers must match session topics  
- 👥 Authors can't present twice in same session
- 📦 Capacity limits per session
- ✅ All papers must be assigned

**The solution?** AI-powered constraint solving!

## Step 1: Define the Problem

Describe the scheduling problem in plain English:

In [1]:
problem_description = '''
Accepted papers

There are fifteen accepted papers. For each paper, we know the short ID, the subject area, the reviewer score (on a five-point scale), and the numeric ID of the presenting author.

P1 has area Computer Vision (CV), score 4.7, and author A1
P2 has area CV, score 4.5, author A2
P3 has area CV, score 4.2, author A3
P4 has area Natural-Language Processing (NLP), score 4.8, author A4
P5 has area NLP, score 4.1, author A1 (same author as P1)
P6 has area Systems (Sys), score 4.3, author A5
P7 has area Reinforcement Learning (RL), score 4.0, author A6
P8 has area Theory, score 3.8, author A7
P9 has area Theory, score 4.4, author A8
P10 has area Sys, score 3.9, author A3 (same author as P3)
P11 has area RL, score 3.7, author A9
P12 has area Sys, score 4.0, author A10
P13 has area NLP, score 3.9, author A11
P14 has area Theory, score 4.6, author A12
P15 has area CV, score 3.9, author A2 (same author as P2)

Conference sessions

The committee has defined six sessions. Each session has three properties: a format (either oral or poster), a capacity (the maximum number of papers it can host), and a topic whitelist (the list of subject areas permitted in that session).

Session OM-A is an oral session with a capacity of 2 papers. It allows only papers in CV.
Session OM-B is an oral session with a capacity of 2 papers. It allows only papers in RL.
Session OA-A is an oral session with a capacity of 2 papers. It allows only papers in NLP.
Session OA-B is an oral session with a capacity of 2 papers. It allows papers in Systems and Theory.
Session Poster-1 is a poster session with a capacity of 4 papers. It allows papers in CV, Systems, and RL.
Session Poster-2 is a poster session with a capacity of 4 papers. It allows papers in NLP and Theory.

Rules that every valid schedule must satisfy

Oral-talk eligibility rule: A paper may be placed in an oral session only if its reviewer score is at least 4.0.

Topic-match rule: A paper can only be assigned to a session if its subject area appears in the session’s topic whitelist.

Capacity rule: A session cannot be assigned more papers than its capacity allows.

Author-session rule: No author may present more than one paper in the same session.
For example, if author A1 is scheduled to present paper P1 in session OM-A, then their other paper P5 must be placed in a different session, such as OA-A or Poster-2, but not also in OM-A.

Coverage requirement: All fifteen papers must be assigned, each to exactly one session. Every assignment must satisfy all of the rules above.
'''

print("✓ Problem defined")

✓ Problem defined


## Step 2: Let CARNAP Solve It

The framework should automatically:
- Identify this as a Constraint Satisfaction Problem (CSP)
- Translate the problem into MiniZinc code
- Find a valid assignment by running the code

Let's first load the config file and initialize an LLM

In [2]:
import yaml
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import Planner, TracePersister
from helpers.trace_explainer import why
# Load config and initialize
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
azure_config = config['api_config']['gpt-4o-azure']
llm = AzureOpenAIGenerator(
    model_name=azure_config['model_name'],
    api_key=azure_config['api_key'],
    model_version=azure_config['openai_api_version'],
    azure_endpoint=azure_config['azure_endpoint']
)

/idiap/temp/lxu/miniconda3/envs/carnap_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



We now pass our problem to a planner:
- that recognizes the problem type
- and devises a plan

In [3]:
router = Planner(generator=llm)
plans, memory, problem_ids = router({"problem": problem_description})
plan = plans[0]


2025-11-07 20:21:08,383 | INFO | Registered agent '<PLAN_START>'
2025-11-07 20:21:08,384 | INFO | Registered agent '<PLAN_END>'
2025-11-07 20:21:08,385 | INFO | Registered agent 'lp_solver'
2025-11-07 20:21:08,385 | INFO | Registered agent 'fol_solver'
2025-11-07 20:21:08,385 | INFO | Registered agent 'csp_solver'
2025-11-07 20:21:08,386 | INFO | Registered agent 'smt_solver'
2025-11-07 20:21:08,386 | INFO | Registered agent 'ilp_solver'
2025-11-07 20:21:08,387 | INFO | Registered agent 'epistemic_solver'
2025-11-07 20:21:08,387 | INFO | Registered agent 'risk_solver'
2025-11-07 20:21:08,387 | INFO | Registered agent 'compositional_solver'
2025-11-07 20:21:08,388 | INFO | Registered agent 'causal_solver'
/remote/idiap.svm/temp.rea01/lxu/251009_Carnap_clean/workshop_release/adaptive_symbolic_reasoning/agents/generation/api.py:203: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instea

Finally, we execute the plan.

The LLM is called to formalize the problem into some formal code. The code is then passed to dedicated solver. 

In [4]:
trace_logger = TracePersister()
plan.execute(memory, trace_logger)
result = memory.read(f"result_{problem_ids[0]}")
print(result["assignments"])


2025-11-07 20:21:16,494 | INFO | Execute plan with topological order...
2025-11-07 20:21:16,495 | INFO | Trace saved: <PLAN_START> → None
2025-11-07 20:21:16,496 | INFO | Trace saved: ques_1 → None
2025-11-07 20:21:21,800 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 20:21:21,801 | INFO | Scratchpad WRITE minizinc_code_csp_solver=include "globals.mzn";

/* ENUMERATIONS */
enum PAPER = {P1, (ttl=None)
2025-11-07 20:21:21,802 | INFO | Attempt 1/1
2025-11-07 20:21:21,803 | INFO | Running command: minizinc --solver gecode /tmp/tmpx56czch2.mzn
2025-11-07 20:21:22,098 | INFO | MiniZinc stdout:
paper_session = [P1: Poster_1, P2: OM_A, P3: OM_A, P4: OA_A, P5: OA_A, P6: OA_B, P7: OM_B, P8: Poster_2, P9: Poster_2, P10: Poster_1, P11: Poster_1, P12: OA_B, P13: Poster_2, P14: Poster_2, P15: Poster_1];
----------

2025-11-07 20:21:22,099 | INFO | Parsed 1 solutions
2025-11-07

{'paper_session': {'P1': 'Poster_1', 'P2': 'OM_A', 'P3': 'OM_A', 'P4': 'OA_A', 'P5': 'OA_A', 'P6': 'OA_B', 'P7': 'OM_B', 'P8': 'Poster_2', 'P9': 'Poster_2', 'P10': 'Poster_1', 'P11': 'Poster_1', 'P12': 'OA_B', 'P13': 'Poster_2', 'P14': 'Poster_2', 'P15': 'Poster_1'}}


## Step 3: Inspect Generated Constraints

As the organizer, **verify all rules were correctly captured** in formal constraints:

In [23]:
print(result['minizinc_code'])

include "globals.mzn";

/* ENUMERATIONS */
enum PAPER = {P1, P2, P3, P4, P5, P6, P7, P8, P9, P10, P11, P12, P13, P14, P15};
enum SUBJECT_AREA = {CV, NLP, Sys, RL, Theory};
enum AUTHOR = {A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12};
enum SESSION = {OM_A, OM_B, OA_A, OA_B, Poster_1, Poster_2};

/* Paper Properties */
array[PAPER] of SUBJECT_AREA: paper_area = [CV, CV, CV, NLP, NLP, Sys, RL, Theory, Theory, Sys, RL, Sys, NLP, Theory, CV];
array[PAPER] of float: paper_score = [4.7, 4.5, 4.2, 4.8, 4.1, 4.3, 4.0, 3.8, 4.4, 3.9, 3.7, 4.0, 3.9, 4.6, 3.9];
array[PAPER] of AUTHOR: paper_author = [A1, A2, A3, A4, A1, A5, A6, A7, A8, A3, A9, A10, A11, A12, A2];

/* Session Properties */
array[SESSION] of int: session_capacity = [2, 2, 2, 2, 4, 4];
array[SESSION] of set of SUBJECT_AREA: session_whitelist = [
  {CV},          % OM_A
  {RL},          % OM_B
  {NLP},         % OA_A
  {Sys, Theory}, % OA_B
  {CV, Sys, RL}, % Poster_1
  {NLP, Theory}  % Poster_2
];

/* Session Format (Booleans fo

## Generate "Why" HTML
#### Understanding Your Results

The output above shows **three predictions** from CARNAP—one for each healthcare task (clinical trial matching, staff credentialing, discharge readiness).

#### Trace Analysis

Each prediction is backed by **generated symbolic code** visible in the trace above, showing the formal logic representations and solver-specific code (Z3, Prover9, or Pyke). However, manually examining traces is cumbersome.

#### The `why()` Function

We've created a helper to make this effortless:

This uses an LLM call to format the trace and predictions into clean, interactive HTML that links each answer to its reasoning chain.

**Next step:** Run `why()` to see a beautiful explanation of how CARNAP arrived at each answer!


In [5]:
from IPython.display import HTML
from helpers.trace_explainer import why

result_summary = []
for pid in problem_ids:
    result = memory.read(f"result_{pid}") or {}
    result_summary.append({
        "problem_id": pid,
        "assignments": result.get("assignments"),
        "parsed_answer": result.get("parsed_answer"),
        "solver_name": result.get("solver_name"),
        "minizinc_code": result.get("minizinc_code"),
    })

metadata = {
    "scenario": "Conference scheduling CSP",
    "problem_ids": problem_ids,
}

narrative_context = (
    "Carnap spotted the description as a scheduling CSP, auto-formalised it into MiniZinc, and solved for a full assignment."
)
style_hint = (
    "Use cards for constraints (capacity, topic match, oral eligibility, author/session) and show the final assignment table prominently."
)
extra_guidance = (
    "Surface the generated MiniZinc code and highlight which constraints guaranteed feasibility. Mention MiniZinc explicitly."
)
html = why(
    llm,
    plan=plan,
    tracer=trace_logger,
    memory=memory,
    metadata=metadata,
    result_summary=result_summary,
    problem_statement=problem_description,
    narrative_context=narrative_context,
    style_hint=style_hint,
    extra_guidance=extra_guidance,
    output_path="demo_paper_assignment.html",
    model_args={"temperature": 0.15, "max_tokens": 3000},
)



2025-11-07 20:21:29,411 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


In [6]:
HTML(html)

Agent,Goal
<PLAN_START>,"Special control marker, indicating the start of the whole working plan."
ques_1,Virtual agent representing ques_1
csp_solver,"A solver tailored for constraint satisfaction problems (CSP) where the goal is to assign values to variables within finite domains such that all explicit or implicit constraints are satisfied. It focuses on configuration, scheduling, and allocation tasks."
<PLAN_END>,"Special control marker, indicating the end of the whole working plan."
Paper ID,Assigned Session
P1,Poster_1
P2,OM_A
P3,OM_A
P4,OA_A
P5,OA_A
